In [1]:
import random

import numpy as np
import pandas as pd

from nltk.corpus import words

In [2]:
from bluegraph import PandasPGFrame
from bluegraph.preprocess import ScikitLearnPGEncoder
from bluegraph.backends.stellargraph import StellarGraphNodeEmbedder

In [3]:
nodes = [
    "Alice", "Bob", "Eric", "John", "Anna", "Laura", "Matt"
]
age = [25, 9, 70, 42, 26, 35, 36]
height = [180, 122, 173, 194, 172, 156, 177]
weight = [75, 43, 68, 82, 70, 59, 81]
sources = [
    "Alice", "Alice", "Bob", "Bob", "Bob", "Eric", "Anna", "Anna", "Matt"
]
targets = [
    "Bob", "Eric", "Eric", "John", "Anna", "Anna", "Laura", "John", "John"
]
weights = [1.0, 2.2, 0.3, 4.1, 1.5, 21.0, 1.0, 2.5, 7.5]
edges = list(zip(sources, targets))
frame = PandasPGFrame(nodes=nodes, edges=edges)

# Add properties

a = pd.DataFrame()
frame.add_node_properties(
    {
        "@id": nodes,
        "age": age
    }, prop_type="numeric")
frame.add_node_properties(
    {
        "@id": nodes,
        "height": height
    }, prop_type="numeric")
frame.add_node_properties(
    {
        "@id": nodes,
        "weight": weight
    }, prop_type="numeric")

edge_weight = pd.DataFrame({
    "@source_id": sources,
    "@target_id": targets,
    "distance": weights
})
frame.add_edge_properties(edge_weight, prop_type="numeric")

In [4]:
props = ["age", "height", "weight"]

In [5]:
df = frame._nodes[props]

In [6]:
df

,age,height,weight
@id,,,
Alice,25,180,75
Bob,9,122,43
Eric,70,173,68
John,42,194,82
Anna,26,172,70
Laura,35,156,59
Matt,36,177,81


In [7]:
df.to_numpy().tolist()

[[25, 180, 75],
 [9, 122, 43],
 [70, 173, 68],
 [42, 194, 82],
 [26, 172, 70],
 [35, 156, 59],
 [36, 177, 81]]

In [8]:
df["_generated_features"] = df.to_numpy().tolist()

In [9]:
df

,age,height,weight,_generated_features
@id,,,,
Alice,25,180,75,"[25, 180, 75]"
Bob,9,122,43,"[9, 122, 43]"
Eric,70,173,68,"[70, 173, 68]"
John,42,194,82,"[42, 194, 82]"
Anna,26,172,70,"[26, 172, 70]"
Laura,35,156,59,"[35, 156, 59]"
Matt,36,177,81,"[36, 177, 81]"


In [10]:
def generate_targets(nodes, s, density=0.2):
    edges = []
    for t in nodes:
        if s < t:
            edge = np.random.choice([0, 1], p=[1 - density, density])
            if edge:
                
                edges.append([s, t])
    return edges


def random_pgframe(n_nodes, density):
    nodes = list(range(n_nodes))

    edges = sum(
        map(lambda x: generate_targets(nodes, x, density), nodes), [])
    edges = pd.DataFrame(
        edges, columns=["@source_id", "@target_id"])
    edges_df = edges.set_index(["@source_id", "@target_id"])
    frame = PandasPGFrame(nodes=nodes, edges=edges_df.index)
    return frame

In [11]:
N = 70
density = 0.13

# Generate a random graph

In [12]:
graph_frame = random_pgframe(N, density)

In [13]:
graph_frame._nodes.reset_index().to_dict("records")

[{'@id': 0},
 {'@id': 1},
 {'@id': 2},
 {'@id': 3},
 {'@id': 4},
 {'@id': 5},
 {'@id': 6},
 {'@id': 7},
 {'@id': 8},
 {'@id': 9},
 {'@id': 10},
 {'@id': 11},
 {'@id': 12},
 {'@id': 13},
 {'@id': 14},
 {'@id': 15},
 {'@id': 16},
 {'@id': 17},
 {'@id': 18},
 {'@id': 19},
 {'@id': 20},
 {'@id': 21},
 {'@id': 22},
 {'@id': 23},
 {'@id': 24},
 {'@id': 25},
 {'@id': 26},
 {'@id': 27},
 {'@id': 28},
 {'@id': 29},
 {'@id': 30},
 {'@id': 31},
 {'@id': 32},
 {'@id': 33},
 {'@id': 34},
 {'@id': 35},
 {'@id': 36},
 {'@id': 37},
 {'@id': 38},
 {'@id': 39},
 {'@id': 40},
 {'@id': 41},
 {'@id': 42},
 {'@id': 43},
 {'@id': 44},
 {'@id': 45},
 {'@id': 46},
 {'@id': 47},
 {'@id': 48},
 {'@id': 49},
 {'@id': 50},
 {'@id': 51},
 {'@id': 52},
 {'@id': 53},
 {'@id': 54},
 {'@id': 55},
 {'@id': 56},
 {'@id': 57},
 {'@id': 58},
 {'@id': 59},
 {'@id': 60},
 {'@id': 61},
 {'@id': 62},
 {'@id': 63},
 {'@id': 64},
 {'@id': 65},
 {'@id': 66},
 {'@id': 67},
 {'@id': 68},
 {'@id': 69}]

In [14]:
graph_frame._edges

Empty DataFrame
Columns: []
Index: [(0, 4), (0, 5), (0, 31), (0, 36), (0, 37), (0, 42), (0, 53), (0, 67), (1, 29), (1, 35), (1, 36), (1, 42), (1, 46), (1, 50), (1, 66), (2, 4), (2, 17), (2, 22), (2, 26), (2, 34), (2, 40), (2, 56), (2, 62), (3, 7), (3, 17), (3, 19), (3, 26), (3, 32), (3, 42), (3, 47), (3, 53), (3, 67), (3, 68), (3, 69), (4, 5), (4, 8), (4, 12), (4, 15), (4, 31), (4, 32), (4, 36), (4, 39), (4, 63), (5, 13), (5, 31), (5, 36), (5, 51), (5, 55), (5, 57), (5, 61), (6, 23), (6, 25), (6, 28), (6, 31), (6, 40), (6, 59), (6, 60), (6, 66), (7, 35), (7, 56), (7, 67), (8, 22), (8, 24), (8, 29), (8, 37), (8, 41), (8, 53), (9, 14), (9, 17), (9, 27), (9, 41), (10, 16), (10, 21), (10, 24), (10, 29), (10, 35), (10, 40), (10, 45), (10, 50), (10, 66), (11, 13), (11, 27), (11, 29), (11, 35), (11, 37), (11, 39), (11, 47), (11, 55), (11, 58), (11, 60), (11, 65), (12, 18), (12, 19), (12, 20), (12, 23), (12, 34), (12, 58), (12, 59), (12, 66), (13, 31), ...]

[304 rows x 0 columns]

# Add node and edge types

In [15]:
types = ["Apple", "Orange", "Carrot"]

In [16]:
node_types = {
    n: np.random.choice(types, p=[0.5, 0.4, 0.1])
    for n in range(N)
}

In [17]:
graph_frame.add_node_types(node_types)

In [18]:
graph_frame._nodes

,@type
@id,
0,Orange
1,Orange
2,Orange
3,Apple
4,Apple
...,...
65,Apple
66,Apple
67,Apple


In [19]:
types = ["isFriend", "isEnemy"]

In [20]:
edge_types = {
    e: np.random.choice(types, p=[0.8, 0.2])
    for e in graph_frame.edges()
}

In [21]:
graph_frame.add_edge_types(edge_types)

In [22]:
graph_frame._edges

@type
@source_id @target_id          
0          4           isFriend
           5           isFriend
           31          isFriend
           36           isEnemy
           37          isFriend
...                         ...
62         66          isFriend
63         65          isFriend
           68           isEnemy
           69          isFriend
65         66          isFriend

[304 rows x 1 columns]

# Add node and edge properties

numerical, categorical, text

## Add node properties

In [23]:
weight = pd.DataFrame(
    [
        (n, np.random.normal(loc=35, scale=5))
        for n in graph_frame.nodes()
    ], 
    columns=["@id", "weight"]
)

In [24]:
graph_frame.add_node_properties(weight, prop_type="numeric")

In [25]:
colors = ["red", "green", "blue"]

In [26]:
colors = pd.DataFrame(
    [
        (n, np.random.choice(colors))
        for n in graph_frame.nodes()
    ], 
    columns=["@id", "color"]
)

In [27]:
graph_frame.add_node_properties(colors, prop_type="category")

In [28]:
desc = pd.DataFrame(
    [
        (n, ' '.join(random.sample(words.words(), 20)))
        for n in graph_frame.nodes()
    ], 
    columns=["@id", "desc"]
)

In [29]:
graph_frame.add_node_properties(desc, prop_type="text")

In [30]:
graph_frame._nodes

,@type,weight,color,desc
@id,,,,
0,Orange,36.073461,red,ukase sialolithiasis downhanging intermediaten...
1,Orange,35.659915,blue,unreplaceable Sobranje foremanship matrilinear...
2,Orange,25.985632,red,linenette exocataphoria elytrum locellate Cerb...
3,Apple,36.389999,green,pentecoster glycosuric sulfurize synderesis an...
4,Apple,34.139613,green,essexite adoptive Marlena fliting disappear mi...
...,...,...,...,...
65,Apple,32.983873,red,hyperpatriotic hematocyte nonecompense submers...
66,Apple,29.692644,green,Eudorina orthosilicate Thalamiflorae rumenocen...
67,Apple,43.656535,red,photooxidation oversweeten fluviomarine impari...


## Add edge properties

In [31]:
years = pd.DataFrame(
    [
        (s, t, np.random.randint(0, 20))
        for s, t in graph_frame.edges()
    ], 
    columns=["@source_id", "@target_id", "n_years"]
)

In [32]:
graph_frame.add_edge_properties(years, prop_type="numeric")

In [33]:
shapes = ["dashed", "dotted", "solid"]
shapes = pd.DataFrame(
    [
        (s, t, np.random.choice(shapes))
        for s, t, in graph_frame.edges()
    ], 
    columns=["@source_id", "@target_id", "shapes"]
)

In [34]:
graph_frame.add_edge_properties(shapes, prop_type="category")

In [ ]:
desc = pd.DataFrame(
    [
        (s, t, ' '.join(random.sample(words.words(), 20)))
        for s, t, in graph_frame.edges()
    ], 
    columns=["@source_id", "@target_id", "desc"]
)

In [ ]:
graph_frame.add_edge_properties(desc, prop_type="text")

In [ ]:
graph_frame._edges

In [ ]:
graph_frame._edge_prop_types

# Property encoding

In [ ]:
## Run this if nltk error occurs (need to download the 'words' corpus)
# import nltk
# nltk.download('words')

In [ ]:
hom_encoder = ScikitLearnPGEncoder(
    node_properties=["weight", "color", "desc"],
    edge_properties=["n_years", "shapes", "desc"],
    edge_features=True,
    heterogeneous=False,
    encode_types=True, drop_types=True, text_encoding="tfidf")

In [ ]:
transformed_frame = hom_encoder.fit_transform(graph_frame)

In [ ]:
transformed_frame._nodes.sample(5)

In [ ]:
hom_encoder._node_encoders

In [ ]:
transformed_frame._edges.sample(5)

In [ ]:
hom_encoder._edge_encoders

## Convert PGFrames into json-ld

In [ ]:
jsonld_repr = graph_frame.to_jsonld(edges_key="outEdges")

In [ ]:
jsonld_repr

In [ ]:
new_frame = PandasPGFrame()
new_frame.from_jsonld(jsonld_repr, types_from_relations=False)

In [ ]:
new_frame._nodes.sample(5)

In [ ]:
new_frame._edges.sample(5)